# Pandas and SQL
Structured Query Language, or **SQL**, is the language of databases. Though "SQL" may seem new to you, you've actually used it in your GIS work, specifically in selecting records by attribute values. SQL, however, is capable of much more than just selecting records; SQL is used to create, read, relate (i.e. join), update, delete records stored in database tables. 

In this exercise we use Pandas to mimic many SQL functions, showing that we don't need to learn daunting database management systems like Oracle, Microsoft SQL Server, PostgreSQL, or even MS Access, to run some of the everyday database functions you might want to run. Instead, we can import data as a dataframe and use some Pandas functions to get what you need. 

Here we explore some useful data query and transformation techniques are applied to marine species observations data extracted from OBIS/SeaMap. These data are stored in the data folder as `Seamap400.csv`. 

→ *You may want to examine this file in Excel to familiarize yourself with the data first.*

In [5]:
#Import the modules
import pandas as pd
import numpy as np #Well need NumPy for some of it's data types...

## Loading the csv data into a Pandas dataframe
We've loaded CSV data into a Pandas dataframe in previous exercises. The code block below demonstrates how the `read_csv` function easily does this. 

In [6]:
#Import the data into a dataframe called "dfSeamap"
dfSeamap = pd.read_csv('../Data/Seamap400.csv')
#Show the first 2 records in the result
dfSeamap.head(2)

,ID,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
0,1,400,1114,6/15/2007 10:06:59,37.27,-75.20,BALA,180522,2,3/13/2007 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
1,2,400,1006,6/15/2007 10:06:59,36.95,-75.87,Bph,180527,2,2/28/2007 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


The result *looks* good, and indeed we can do much with this dataframe. BUT, there are some "gotchas" that we can fix:
* First, Pandas assigns it's own index, when we might want to keep the ID field as the index values.
* The `dataset` and `rowid` fields, are actually nominal values, not actual numbers, and we should import these as strings, not numbers (in case they have a leading zero, for example).
* The `obs_date` field is imported as a string, but we can convert this as a date object, which will allow for selection via time slices.


So... to fix this, we can specify a few extra parameters when importing the data. Below is the `read_csv()` function applies as above, but with three additional parameters:
* The `index_col` specifies which column will be our dataFrame's **index**. 
* `parse_dates` indicates that the `last_mod` and `obs_date` columns are a **date fields**, not strings, which enables us to to time-based queries. 
* And finally, the `dtype` functions allows us to override the defaut data type assignments. Here, we pass a dictionary consisting of column names (keys) and data types (values) to indicate which columns:data type pairs we want to specifiy. 

In [41]:
#Import the Seamap400.csv file to a DataFrame
dfSeamap = pd.read_csv('../Data/Seamap400.csv', 
                       index_col = 'ID',
                       parse_dates=['last_mod','obs_date'],
                       dtype={'datasetid':'str',
                              'rowid':'str'
                             }
                      )
dfSeamap.head(2)

,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
ID,,,,,,,,,,,,,
1,400,1114,2007-06-15 10:06:59,37.27,-75.20,BALA,180522,2,2007-03-13 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington
2,400,1006,2007-06-15 10:06:59,36.95,-75.87,Bph,180527,2,2007-02-28 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington


Other than the ID now being used as the index the result looks the same, but if you inspect the data types, you'll see we got what we wanted.

In [42]:
#Check the data types of the import
dfSeamap.dtypes

datasetid             object
rowid                 object
last_mod      datetime64[ns]
latitude             float64
longitude            float64
sp_code               object
sp_tsn                 int64
sp_count               int64
obs_date      datetime64[ns]
scientific            object
sp_common             object
sp_class              object
provider              object
dtype: object

## Exploring our dataFrame
Now, let's examine how many unique species there are. This is analogous to the `SELECT DISTINCT` SQL operation.

In [10]:
dfSeamap.sp_common.unique()

array(['rorquals', 'Fin Whale', 'Loggerhead', 'cartilaginous fishes',
       'Basking Shark', 'Common Dolphin', 'dolphins',
       'Northern Right Whale', 'Short-finned Pilot Whale',
       'Ocean Sunfish', 'Atlantic Ridley', 'Atlantic Manta',
       'Humpback Whale', 'cow-nose rays', 'Bottlenose Dolphin',
       'Sea Turtles'], dtype=object)

## Selecting columns of data from a dataFrame
Selecting columns is analogous to the SQL `SELECT ... FROM` clause, but the syntax in Pandas is a bit different.

#### ♦ Selecting data from <u>a single column</u> in the dataframe
Selecting a single column is easy as specifying the column name, either between brackets with the field name in quotes, or using the dot notation. (*The former is preferred, in case your field name is the the same as another property or method of a dataFrame.*)

When only one column is selected, the object returned is actually not a dataFrame, but a pandas **series** object, which is quite similar to a one-dimensional NumPy array. (However, in Pandas, series are limited to one dimension...)

In [14]:
#Select just the sp_common field into a new view
dfSelect = dfSeamap['sp_common']
dfSelect.head()

0     rorquals
1    Fin Whale
2    Fin Whale
3    Fin Whale
4    Fin Whale
Name: sp_common, dtype: object

In [12]:
#Using "dot notation" (<dataframe>.<field name>) also works
commonNames = dfSeamap.sp_common
type(commonNames)

pandas.core.series.Series

In [15]:
#As the object returned is a series, we can get records from its index
commonNames[:8] #Returns the first 8 records

0     rorquals
1    Fin Whale
2    Fin Whale
3    Fin Whale
4    Fin Whale
5    Fin Whale
6    Fin Whale
7    Fin Whale
Name: sp_common, dtype: object

In [17]:
#What code would you use to get the last 8 records? (Replace the ???)
commonNames[???]

2264    Sea Turtles
2265    Sea Turtles
2266    Sea Turtles
2267    Sea Turtles
2268    Sea Turtles
Name: sp_common, dtype: object

#### ♦ Selecting data  from <u>multiple</u> columns
Selecting multiple columns can be done by including a list of the column names. *Note the double set of brackets: the outer set is the syntax of the dataFrame object, and the inner set denotes the list of field names we want.*

Here, the object returned resembles a dataFrame, not a series (since Pandas' series can only be one-dimension). However, in actuality, this is simply a **view** of the original dataFrame, not a new object - much like query views in SQL...

In [20]:
#Select the 'sp_common' and 'sp_class' columns into a new view
dfSelect = dfSeamap[['sp_common','sp_class']]
dfSelect.head()

,sp_common,sp_class
0,rorquals,Mammalia
1,Fin Whale,Mammalia
2,Fin Whale,Mammalia
3,Fin Whale,Mammalia
4,Fin Whale,Mammalia


In [22]:
##What code would create a table of just the species common name and the latitude and longitude columns?
dfSelect2 = dfSeamap[['sp_common','latitude','longitude']]
dfSelect2.head()

,sp_common,latitude,longitude
0,rorquals,37.27,-75.20
1,Fin Whale,36.95,-75.87
2,Fin Whale,36.96,-75.43
3,Fin Whale,36.95,-75.49
4,Fin Whale,36.70,-75.55


## Selecting rows in a dataFrame
Selecting rows, analogous to the `select...where` statement in SQL, can be done by creating **boolean masks** of the criteria you want and then applying those masks. This can be done explicitily as two steps, or more commonly in a single, compount step. 

Before making our selection, let's examine how we can easily extract a list of valid options using the `unique` property.

In [23]:
#List the unique values in the 'sp_class' field.
dfSeamap['sp_class'].unique()

array(['Mammalia', 'Reptilia', 'Chondrichthyes', 'Actinopterygii'],
      dtype=object)

In [26]:
#Display just the *number* of unique latitude values
dfSeamap['latitude'].nunique()

267

#### Selecting rows in two steps:

In [27]:
#Step 1: Create the boolean mask
theMask = dfSeamap['sp_class'] == 'Mammalia'
theMask[2150:2156] #Show a selection of records, to display values

2150     True
2151     True
2152     True
2153    False
2154    False
2155    False
Name: sp_class, dtype: bool

In [31]:
#Step 2: Applying the mask to return only 'true' records
dfMammals = dfSeamap[theMask]
dfMammals.sample(4) #Show a random sample from the product

,ID,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider
1166,1167,400,487,6/15/2007 10:06:59,34.67,-76.69,DELP,180415,12,1/24/2007 14:26:00,Delphinidae,dolphins,Mammalia,University of North Carolina at Wilmington
1332,1333,400,76,6/15/2007 10:06:59,35.40,-75.44,Ttr,180426,12,12/11/2006 13:53:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
1385,1386,400,258,6/15/2007 10:06:59,34.61,-76.42,Ttr,180426,1,12/18/2006 13:05:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington
2028,2029,400,1920,6/15/2007 10:06:59,35.21,-75.55,Ttr,180426,4,4/23/2007 10:17:00,Tursiops truncatus,Bottlenose Dolphin,Mammalia,University of North Carolina at Wilmington


In [30]:
#Proof that we only got mammals in the result
dfMammals['sp_class'].unique()

array(['Mammalia'], dtype=object)

#### Selecting rows in one step:

In [37]:
#Get all the mammals and report all unique scientific names
dfMammals = dfSeamap[dfSeamap['sp_class'] == 'Mammalia']
dfMammals.shape #Reveal the size of the result

(929, 14)

In [32]:
dfMammals['scientific'].unique()

array(['Balaenopteridae', 'Balaenoptera physalus', 'Delphinus delphis',
       'Delphinidae', 'Eubalaena glacialis', 'Globicephala macrorhynchus',
       'Megaptera novaeangliae', 'Tursiops truncatus'], dtype=object)

#### Selecting rows using `query`:
We can also execute familiar **query strings** to extract records. *Personally, I don't like this method as the query strings tend to be finicky and can be tough to use with quotes..*

In [35]:
dfMammals = dfSeamap.query("sp_class == 'Mammalia'")
dfMammals.shape

(929, 14)

#### Complex row selections
We can combine masks using logical operators (`&` = "and", `|` = "or").

In [43]:
#Select mammal records in the 2nd half of 2006
mammalMask = dfSeamap['sp_class'] == 'Mammalia'

#Create date objects for the start and end dates
startDate = np.datetime64('2006-07-01')
endDate = np.datetime64('2007-01-01')

#Create the date masks
startMask = (dfSeamap['obs_date'] >= startDate) 
endMask = (dfSeamap['obs_date'] < endDate)

#Apply the masks, using the bitwise '&' to return rows where all masks are true
dfSelect2 = dfSeamap[mammalMask & startMask & endMask]
dfSelect2['scientific'].unique()

array(['Delphinus delphis', 'Delphinidae', 'Eubalaena glacialis',
       'Tursiops truncatus'], dtype=object)

#### Return rows matching a substring
The `.str` function on a column allows us to use some string operations  on the values in that field.  Here we use the string `startswith` function to return all rows where the row's value starts with 'Delphin'. See https://pandas.pydata.org/pandas-docs/stable/text.html for other string operations.

In [44]:
#Select rows where the scientific name starts with "Delphin" 
dolphinMask = dfSeamap['scientific'].str.startswith('Delphin')
dfDolphins = dfSeamap[dolphinMask]

#Use the nunique function to just return the number 
#  of unique scientific names
dfDolphins['scientific'].nunique()

2

#### More complex queries with `apply` and  `lambda`
For maximum flexibility, we can actually write our own functions to be applied to each value in a column (or multiple columns). This is done using the `apply` function to a dataFrame and then specifying the subcode we want to use with Python's `lambda` statement. (This seems fairly complex at first, but it actually somewhat straightforward -- and can be very useful...)

In [45]:
#Create a mask by searching each row for the string "Whale" and apply the mask
# to list the scientific names of these records
whaleMask = dfSeamap['sp_common'].apply(lambda x: 'Whale' in x)
dfWhale = dfSeamap[whaleMask]
dfWhale['scientific'].unique()

array(['Balaenoptera physalus', 'Eubalaena glacialis',
       'Globicephala macrorhynchus', 'Megaptera novaeangliae'],
      dtype=object)

## Grouping/Aggregating data
Pandas can aggregate data on values like SQL as well. We do this with the `groupby` statement

In [46]:
#Count the observations by common name
grpSpCommon = dfSeamap.groupby('sp_common')
grpSpCommon['sp_common'].count()

sp_common
Atlantic Manta               24
Atlantic Ridley               5
Basking Shark                29
Bottlenose Dolphin          849
Common Dolphin               16
Fin Whale                     8
Humpback Whale                4
Loggerhead                  957
Northern Right Whale         11
Ocean Sunfish                64
Sea Turtles                 116
Short-finned Pilot Whale      2
cartilaginous fishes        143
cow-nose rays                 2
dolphins                     38
rorquals                      1
Name: sp_common, dtype: int64

We can also show *all* the summary stats with the dataFrame's `describe` function.

In [47]:
grpSpCommon.describe()

latitude                                       \
                            count       mean       std    min      25%   
sp_common                                                                
Atlantic Manta               24.0  34.748750  0.444936  34.20  34.3100   
Atlantic Ridley               5.0  34.630000  0.955432  33.52  33.7100   
Basking Shark                29.0  35.035172  0.372948  34.14  34.8200   
Bottlenose Dolphin          849.0  35.090942  0.828578  33.41  34.5200   
Common Dolphin               16.0  34.958125  0.520861  33.45  34.7175   
Fin Whale                     8.0  37.025000  0.359007  36.56  36.8875   
Humpback Whale                4.0  35.582500  0.737852  34.93  35.2225   
Loggerhead                  957.0  34.981139  0.674923  33.40  34.5100   
Northern Right Whale         11.0  35.228182  0.995910  33.48  35.1050   
Ocean Sunfish                64.0  35.012344  0.911255  33.42  34.3200   
Sea Turtles                 116.0  34.852328  0.698799  33.46  34.3900   
Short-finned Pilot Whale      2.0  34.850000  0.028284  34.83  34.8400   
cartilaginous fishes        143.0  35.039371  0.556768  33.53  34.7350   
cow-nose rays                 2.0  35.020000  1.187939  34.18  34.6000   
dolphins                     38.0  34.961053  1.095331  33.52  34.1300   
rorquals                      1.0  37.270000       NaN  37.27  37.2700   

                                                 longitude             \
                             50%      75%    max     count       mean   
sp_common                                                               
Atlantic Manta            34.710  35.0475  35.60      24.0 -75.676250   
Atlantic Ridley           35.100  35.1500  35.67       5.0 -76.208000   
Basking Shark             35.090  35.1900  35.67      29.0 -75.797931   
Bottlenose Dolphin        35.180  35.6000  37.40     849.0 -76.143345   
Common Dolphin            34.950  35.2825  35.67      16.0 -75.427500   
Fin Whale                 36.955  37.1625  37.75       8.0 -75.522500   
Humpback Whale            35.380  35.7400  36.64       4.0 -75.370000   
Loggerhead                34.990  35.4000  37.67     957.0 -75.837492   
Northern Right Whale      35.210  35.4100  36.96      11.0 -75.896364   
Ocean Sunfish             34.995  35.8625  36.66      64.0 -76.030625   
Sea Turtles               34.820  35.1025  37.80     116.0 -76.016724   
Short-finned Pilot Whale  34.850  34.8600  34.87       2.0 -75.180000   
cartilaginous fishes      35.140  35.4000  36.99     143.0 -75.694476   
cow-nose rays             35.020  35.4400  35.86       2.0 -76.625000   
dolphins                  34.735  35.8000  37.52      38.0 -76.296053   
rorquals                  37.270  37.2700  37.27       1.0 -75.200000   

                            ...    sp_count        sp_tsn                 \
                            ...         75%    max  count      mean  std   
sp_common                   ...                                            
Atlantic Manta              ...        1.00    6.0   24.0  160992.0  0.0   
Atlantic Ridley             ...        1.00    2.0    5.0  551770.0  0.0   
Basking Shark               ...        1.00    6.0   29.0  159907.0  0.0   
Bottlenose Dolphin          ...        8.00  225.0  849.0  180426.0  0.0   
Common Dolphin              ...       23.00   60.0   16.0  180438.0  0.0   
Fin Whale                   ...        4.50    6.0    8.0  180527.0  0.0   
Humpback Whale              ...        1.00    1.0    4.0  180530.0  0.0   
Loggerhead                  ...        1.00   18.0  957.0  173830.0  0.0   
Northern Right Whale        ...        2.50    5.0   11.0  180537.0  0.0   
Ocean Sunfish               ...        1.00    3.0   64.0  173414.0  0.0   
Sea Turtles                 ...        1.00    3.0  116.0  173828.0  0.0   
Short-finned Pilot Whale    ...       14.00   16.0    2.0  180466.0  0.0   
cartilaginous fishes        ...        1.00   18.0  143.0  159785.0  0.0   
cow-nose rays 

## Reshaping tables
In addition to grouping data, we can "pivot" the data, summarizing by one field and aggregating values across other fields. Let's look at an example where we display 

In [71]:
#Extract just the dolphin records
#dfDolphin = dfSeamap.query("sp_common == 'dolphins'").copy(deep=True)
dfDolphin = dfSeamap.copy(deep=True)

In [76]:
#Add a "day" column (https://pandas.pydata.org/pandas-docs/stable/api.html#datetimelike-properties)
dfDolphin['day'] = dfDolphin['obs_date'].dt.weekofyear
dfDolphin.head()

,datasetid,rowid,last_mod,latitude,longitude,sp_code,sp_tsn,sp_count,obs_date,scientific,sp_common,sp_class,provider,day
ID,,,,,,,,,,,,,,
1,400,1114,2007-06-15 10:06:59,37.27,-75.20,BALA,180522,2,2007-03-13 14:11:00,Balaenopteridae,rorquals,Mammalia,University of North Carolina at Wilmington,11
2,400,1006,2007-06-15 10:06:59,36.95,-75.87,Bph,180527,2,2007-02-28 10:52:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington,9
3,400,1007,2007-06-15 10:06:59,36.96,-75.43,Bph,180527,3,2007-02-28 11:19:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington,9
4,400,1008,2007-06-15 10:06:59,36.95,-75.49,Bph,180527,6,2007-02-28 11:35:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington,9
5,400,1010,2007-06-15 10:06:59,36.70,-75.55,Bph,180527,3,2007-02-28 13:58:00,Balaenoptera physalus,Fin Whale,Mammalia,University of North Carolina at Wilmington,9


In [77]:
#Pivot to a table listing row=species, col = day of year, values = latitude
dfLat = dfDolphin.pivot_table(index='sp_common',columns='day',values='latitude')
dfLat.head()

day,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,49,50,51
sp_common,,,,,,,,,,,,,,,,,,,,,
Atlantic Manta,NaN,NaN,NaN,NaN,34.260,35.330000,34.528889,NaN,34.716667,NaN,...,34.800000,35.020000,NaN,NaN,34.610000,35.050000,NaN,NaN,35.305000,34.200000
Atlantic Ridley,NaN,33.710000,NaN,NaN,NaN,NaN,35.125000,NaN,NaN,35.670000,...,33.520000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Basking Shark,NaN,NaN,NaN,NaN,NaN,35.097500,34.540000,34.140000,35.670000,NaN,...,35.110909,35.146667,NaN,35.470000,35.670000,35.130000,NaN,NaN,NaN,NaN
Bottlenose Dolphin,33.918333,34.261857,34.386667,34.2895,34.545,35.821563,34.741731,33.813636,35.161167,35.176667,...,35.032673,35.139459,35.22,35.632121,36.354651,35.729038,36.17,33.907895,35.045800,34.192162
Common Dolphin,NaN,34.080000,NaN,NaN,NaN,35.470000,34.845714,NaN,NaN,NaN,...,35.530000,NaN,NaN,35.470000,NaN,NaN,NaN,NaN,35.103333,NaN


## Joining tables
Also like SQL, pandas can join tables. Below we'll create two tables from our aggregated data: one will list the minimum of the latitude and longitude columns, and the second will list the maximum values. Then we'll join these two tables and compute the geographic extent of each species observations. 

In [48]:
#Create dataFrames of the minimum and then maximum of the lat and lng fields
minCoords = grpSpCommon['latitude','longitude'].min()
maxCoord = grpSpCommon['latitude','longitude'].max()

In [49]:
# Have a look at what is returned (for the min table)
minCoords.head()

,latitude,longitude
sp_common,,
Atlantic Manta,34.20,-76.48
Atlantic Ridley,33.52,-77.53
Basking Shark,34.14,-77.31
Bottlenose Dolphin,33.41,-78.40
Common Dolphin,33.45,-77.47


In [50]:
#Use the Pandas 'merge' command to join the two tables
sppExtent = pd.merge(left=minCoords,     #Specifies the left table
                     right=maxCoord,     #Specifies the right table
                     how = 'inner',      #Specifies the type of join
                     left_index=True,    #Use the index of the left table as the join item
                     right_index=True)   #Use the index of the right table as the join item
#Have a look
sppExtent.head()

,latitude_x,longitude_x,latitude_y,longitude_y
sp_common,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86
Atlantic Ridley,33.52,-77.53,35.67,-74.95
Basking Shark,34.14,-77.31,35.67,-75.19
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89
Common Dolphin,33.45,-77.47,35.67,-74.92


In [51]:
#Rename the columns to the values in the list provided
sppExtent.columns = ['minX','minY','maxX','maxY']
sppExtent.head()

,minX,minY,maxX,maxY
sp_common,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86
Atlantic Ridley,33.52,-77.53,35.67,-74.95
Basking Shark,34.14,-77.31,35.67,-75.19
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89
Common Dolphin,33.45,-77.47,35.67,-74.92


In [52]:
#Compute two new columns as the difference between max and min
sppExtent['XRange'] = sppExtent.maxX - sppExtent.minX
sppExtent['YRange'] = sppExtent.maxY - sppExtent.minY
sppExtent.head()

,minX,minY,maxX,maxY,XRange,YRange
sp_common,,,,,,
Atlantic Manta,34.20,-76.48,35.60,-74.86,1.40,1.62
Atlantic Ridley,33.52,-77.53,35.67,-74.95,2.15,2.58
Basking Shark,34.14,-77.31,35.67,-75.19,1.53,2.12
Bottlenose Dolphin,33.41,-78.40,37.40,-74.89,3.99,3.51
Common Dolphin,33.45,-77.47,35.67,-74.92,2.22,2.55
